In [49]:
import os
import numpy as np
import pandas as pd
import yfinance as yf

In [50]:
from enum import Enum

In [51]:
last_year = 2000  # will iterate backwards
working_days = 250  # number of working days in a year
upper_bound = 2020  # won't include this year

In [52]:
folder_path = '../third-party/US Stocks/'

In [53]:
NYSE_PATH, AMEX_PATH, NASDAQ_PATH = 'NYSE.csv', 'AMEX.csv', 'NASDAQ.csv'

In [54]:
def get_tickers():
    def to_list(filepath):
        ticker_list = pd.read_csv(filepath, engine='python')
        return ticker_list[~ticker_list['Symbol'].str.contains("\.|\^")]['Symbol'].tolist()
    # Return the ticks from all available traders
    return (to_list(folder_path + NYSE_PATH), to_list(folder_path + AMEX_PATH), to_list(folder_path + NASDAQ_PATH))

In [7]:
for tick_list in get_tickers():
    for tick in tick_list:
        stock = yf.Ticker(tick)
        stock_history = stock.history(period="max")
        stock_history = stock_history[['Open', 'High', 'Close', 'Volume']]
        stock_history.to_pickle('../third-party/US Stocks/Stocks/%s.pkl' % tick)

- BMY~: No data found, symbol may be delisted
- BEPC$: No data found, symbol may be delisted
- BEP$: No data found, symbol may be delisted
- CTT           : No data found, symbol may be delisted
- CELG~: No data found, symbol may be delisted
- ECOM          : No data found, symbol may be delisted
- ECC           : No data found, symbol may be delisted
- ETX           : No data found, symbol may be delisted
- FEI           : No data found, symbol may be delisted
- GEN           : No data found, symbol may be delisted
- GIX~: No data found, symbol may be delisted
- STAR          : No data found, symbol may be delisted
- NTEST: 1d data not available for startTime=-2208988800 and endTime=1597328199. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CTEST: 1d data not available for startTime=-2208988800 and endTime=1597328255. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SAND          : No data found, symbol may be

- CPAAW: 1d data not available for startTime=-2208988800 and endTime=1597328942. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CREXW: 1d data not available for startTime=-2208988800 and endTime=1597328950. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CRSAW: 1d data not available for startTime=-2208988800 and endTime=1597328952. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CYRXW: 1d data not available for startTime=-2208988800 and endTime=1597328955. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DRIOW: 1d data not available for startTime=-2208988800 and endTime=1597328964. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DSKEW: 1d data not available for startTime=-2208988800 and endTime=1597328965. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DRMT: 1d

- HYMCW: 1d data not available for startTime=-2208988800 and endTime=1597329189. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMACW: 1d data not available for startTime=-2208988800 and endTime=1597329200. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMTXW: 1d data not available for startTime=-2208988800 and endTime=1597329201. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMRNW: 1d data not available for startTime=-2208988800 and endTime=1597329204. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IEAWW: 1d data not available for startTime=-2208988800 and endTime=1597329208. Only 100 years worth of day granularity data are allowed to be fetched per request.
- INSUW: 1d data not available for startTime=-2208988800 and endTime=1597329215. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IGICW: 1

- ROCHW: 1d data not available for startTime=-2208988800 and endTime=1597329550. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SAMAW: 1d data not available for startTime=-2208988800 and endTime=1597329564. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHIPW: 1d data not available for startTime=-2208988800 and endTime=1597329572. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHIPZ: 1d data not available for startTime=-2208988800 and endTime=1597329572. Only 100 years worth of day granularity data are allowed to be fetched per request.
- EYESW: 1d data not available for startTime=-2208988800 and endTime=1597329573. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SCCI: 1d data not available for startTime=-2208988800 and endTime=1597329584. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SGLBW: 1d

In [55]:
blacklist, shared_idx, shared_col, collective_frames = [], {}, {}, {}
for root, dirs, files in os.walk("../third-party/US Stocks/Stocks/"):
    for filename in files:
        stock = pd.read_pickle("%s/%s" % (root, filename))
        lower_limit = (stock.index > pd.to_datetime('%s-01-01' % last_year))
        upper_limit = (stock.index < pd.to_datetime('%d-01-01' % upper_bound))
        x = upper_limit & lower_limit
        if len(stock.index[x]) >= (working_days * (upper_bound - last_year)):
            # Identifying the shared timestamps and shared columns
            shared_idx = set(stock.index[x]) if len(shared_idx) == 0 else set(shared_idx) | set(stock.index[x])
            # Saving the current dataset for later use
            collective_frames[filename] = stock
# Shared indice and columns for the current year
shared_idx = sorted(shared_idx)
# Resulting tensor
for key in list(sorted(collective_frames.keys())):
    try:
        collective_frames[key].reindex(shared_idx).sort_index()
    except:
        blacklist.append(key)

In [56]:
# while True:
shared_idx, shared_col, collective_frames = {}, {}, {}
for root, dirs, files in os.walk("../third-party/US Stocks/Stocks/"):
    for filename in files:
        if filename not in blacklist:
            stock = pd.read_pickle("%s/%s" % (root, filename))
            lower_limit = (stock.index > pd.to_datetime('%s-01-01' % last_year))
            upper_limit = (stock.index < pd.to_datetime('%d-01-01' % upper_bound))
            x = upper_limit & lower_limit
            if len(stock.index[x]) >= (working_days * (upper_bound - last_year)):
                # Identifying the shared timestamps and shared columns
                shared_idx = set(stock.index[x]) if len(shared_idx) == 0 else set(shared_idx) | set(stock.index[x])
                shared_col = set(stock.columns) if len(shared_col) == 0 else set(shared_col) | set(stock.columns)
                # Saving the current dataset for later use
                collective_frames[filename] = stock
            elif len(stock.index) == 0:
                os.remove("%s/%s" % (root, filename))
# if len(collective_frames.keys()) == 0: break  # No year has left
# Verbosing results from each year
print(last_year, len(collective_frames.keys()))
# Shared indice and columns for the current year
shared_idx = sorted(shared_idx)
shared_col = sorted(shared_col)
# Resulting tensor
dataframes = []
for key in list(sorted(collective_frames.keys())):
    dataframe = collective_frames[key].reindex(shared_idx).sort_index()
    dataframe = dataframe[shared_col].fillna(method='ffill')
    dataframe = dataframe.fillna(method='bfill')
    if dataframe.isna().sum().sum() > 0:
        print(key, "FAIL!")
    dataframes.append(dataframe.values)
matrix = np.stack(dataframes)
# Preparing for saving
shared_idx = [str(idx) for idx in shared_idx]
shared_col = [str(idx) for idx in shared_col]
sulfix = "../prepared-data/US Stocks %d:%d" % (last_year, (upper_bound - 1))
# Saving the tensor
np.save(sulfix + " (%dx%dx%d)" % matrix.shape, matrix)
# 0-Axis description
with open(sulfix + " (%dx%dx%d).AX0" % matrix.shape,'w') as f:
    f.write('\n'.join(list(sorted(collective_frames.keys()))))
# 1-Axis description
with open(sulfix + " (%dx%dx%d).AX1" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_idx))
# 2-Axis description
with open(sulfix + " (%dx%dx%d).AX2" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_col))
# last_year -= 1  # Iterating over years

2000 1991
